In [1]:
import pandas as pd 
import numpy as np

import os

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.width', 1000)

In [2]:
FOLDER = './vk_groups'
group_names = {'Мой город Пермь': 'vikiperm', '59.RU': 'news59ru', 'BusinessNews': 'gazetabc'}

In [3]:
def get_vk_group_posts_with_topics(group_name, folder=FOLDER):
    if os.path.isfile(f'{folder}/{group_name}/{group_name}_posts_with_topic.csv'):
        path = f'{folder}/{group_name}/{group_name}_posts_with_topic.csv'
        df = pd.read_csv(path, index_col=0)
        return df
    else:
        print('Файла {group_name}_posts_with_topic.json не существует')

In [4]:
FOLDER_ANALYTICS = './report'
def save_diagram(fig, fig_name, folder=FOLDER_ANALYTICS):
    if os.path.exists(f'{folder}'):
        path = f'{folder}/{fig_name}.html'
        fig.write_html(path)

        path = f'{folder}/{fig_name}.png'
        fig.write_image(path, width=1600, height=720)
    else:
        os.mkdir(FOLDER_ANALYTICS)
        path = f'{folder}/{fig_name}.html'
        fig.write_html(path)

        path = f'{folder}/{fig_name}.png'
        fig.write_image(path, scale=1, width=1600, height=720)

## Загрузка данных

In [5]:
list_df_posts = []
for group_name in group_names.values():
    df_posts = get_vk_group_posts_with_topics(group_name)
    list_df_posts.append(df_posts)

df_posts = pd.concat(list_df_posts)
df_posts.reset_index(drop=True, inplace=True)

In [6]:
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         330 non-null    int64  
 1   text         330 non-null    object 
 2   topic        330 non-null    object 
 3   topic_proba  330 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.4+ KB


Для того чтобы снизить искажение аналитики тем постов, которые предсказала нейросеть <br>
отфильтруем посты с низкой вероятностью прогноза темы.

In [7]:
limit_proba = 0.6
filtr = df_posts['topic_proba'] > limit_proba
df_posts = df_posts.loc[filtr].reset_index(drop=True)

In [8]:
df_posts.shape

(215, 4)

In [9]:
df_posts.sample(n=10)

,date,text,topic,topic_proba
152,1684256460,В [club135161380|Законодательном Собрания Перм...,Экономика,0.613511
46,1684296060,В Перми пройдёт ночной благотворительный забег...,Спорт,0.623267
197,1683613500,В Пермском крае выбирают подрядчиков для ремон...,Город,0.805498
90,1684511415,В центре Перми загорелось заброшенное здание. ...,Культура,0.643033
95,1684504803,"Теплый воздух обдувает волосы, от воды веет св...",Город,0.643358
167,1684054804,К возведению универсальной спортивной арены в ...,Город,0.630398
67,1684590720,За последние сутки в Пермском крае зарегистрир...,Город,0.716685
91,1684508820,"Уже с понедельника, 22 мая, следующая по кольц...",Город,0.938504
186,1683725220,В расписании аэропорта Большое Савино появился...,Город,0.965539
31,1684406760,"Солнечная набережная, май в Перми 🌸\n\nФото [i...",Город,0.964992


In [10]:
# Сортировка постов по дате
df_posts['date'] = pd.to_datetime(df_posts['date'], unit='s').dt.strftime('%Y-%m-%d')

In [11]:
df_posts['date'] = df_posts['date'].astype(dtype='datetime64[D]')

In [12]:
df_posts.sort_values(by='date', ascending=False, inplace=True)
df_posts.reset_index(drop=True, inplace=True)

In [13]:
with pd.option_context('display.max_colwidth', 100):
    print(df_posts.head(10))

        date                                                                                                 text              topic  topic_proba
0 2023-05-21  Доставим за 77 минут или сет роллов - бесплатно! 🍱\n\nКрутая акция в Томми Фиш - доставим ваш за...    Наука и техника     0.642851
1 2023-05-21  Документацию для изъятия в государственную собственность недвижимости с целью строительства втор...              Город     0.690082
2 2023-05-21  Вопрос о том, куда поехать в отпуск, второе лето подряд занимает головы наших соотечественников....          Экономика     0.685514
3 2023-05-21  Мы продолжаем задавать неловкие вопросы врачам. В этом выпуске мы разбираемся в бесконечном голо...          Экономика     0.725272
4 2023-05-21  Все овощи уже рассажены на свои законные места, а свободная грядка всё-таки осталась? Дорогие да...    Наука и техника     0.982598
5 2023-05-21  ФСБ возбудила уголовное дело за госизмену на двоих жителей Пермского края. Ранее их заочно осуди...  Силовые с

In [14]:
df_posts['topic'].unique()

array(['Наука и техника', 'Город', 'Экономика', 'Силовые структуры',
       'Культура', 'Спорт'], dtype=object)

In [15]:
df_posts.groupby(by='date').count()

,text,topic,topic_proba
date,,,
2023-05-07,5,5,5
2023-05-08,7,7,7
2023-05-09,7,7,7
2023-05-10,9,9,9
2023-05-11,4,4,4
2023-05-12,4,4,4
2023-05-13,6,6,6
2023-05-14,5,5,5
2023-05-15,10,10,10


In [16]:
df_posts.drop(columns=['topic_proba'], inplace=True)

## Формирование отчета

В качестве отчета построим столбчатые диаграммы, которые будут отражать <br>
количество постов в день в течение выбранного промежутка

Построим диаграмму количества постов в день

In [22]:
#Отбираем посты за заданный интервал
start_date = '2023-05-07'
end_time = 'today'

filtr = (df_posts['date'] >= start_date) & (df_posts['date'] <= end_time)
df_posts_in_interval = df_posts.loc[filtr]


In [23]:
df_posts_in_interval = df_posts_in_interval.groupby(by=['date'], as_index=False).count()

In [24]:
dates = df_posts_in_interval['date'].dt.strftime('%d/%m')
topics_count = df_posts_in_interval['topic'] 
fig = go.Figure(data=[
    go.Bar(x=dates, y=topics_count, marker_color = 'rgb(26, 118, 255)')])

fig.update_layout(
    font=dict(family='Courier New, monospace', size=20),
    xaxis_title='Дата',
    yaxis_title='Количество постов',
    title=dict(text="Количество постов в ежесуточном разрезе", font=dict(size=26), automargin=True, yref='paper')
)
save_diagram(fig, 'общее_количество_постов_в_день')
fig.show()

Построим диаграмму распределения тем постов в межсуточном разрезе

In [25]:
#Отбираем посты за заданный интервал
start_date = '2023-05-07'
end_time = 'today'

filtr = (df_posts['date'] >= start_date) & (df_posts['date'] <= end_time)
df_topic_slice = df_posts.loc[filtr]

#Группирум данные по дате и теме
df_topic_slice['date'] = df_topic_slice['date'].dt.strftime('%d/%m')
df_topic_slice = df_topic_slice.groupby(by=['date', 'topic'], as_index=False).count()

#преобразуем таблицы, так чтобы в столбцах были названия тем, индексом была дата, а
# значениями было количество постов
df_topic_slice = df_topic_slice.pivot(index=['topic'], columns=['date'], values=['text'])
df_topic_slice.columns = df_topic_slice.columns.droplevel()
df_topic_slice.reset_index(inplace=True)
df_topic_slice.columns.name = None
df_topic_slice

,topic,07/05,08/05,09/05,10/05,11/05,12/05,13/05,14/05,15/05,16/05,17/05,18/05,19/05,20/05,21/05
0,Город,4.0,5.0,5.0,6.0,3.0,2.0,3.0,3.0,9.0,9.0,11.0,15.0,13.0,11.0,3.0
1,Культура,1.0,NaN,2.0,1.0,NaN,2.0,1.0,2.0,NaN,4.0,4.0,5.0,12.0,8.0,3.0
2,Наука и техника,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,5.0,2.0,5.0,3.0
3,Силовые структуры,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,3.0,5.0,6.0,3.0
4,Спорт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,3.0,1.0,NaN
5,Экономика,NaN,1.0,NaN,2.0,NaN,NaN,1.0,NaN,1.0,3.0,3.0,5.0,3.0,2.0,3.0


In [26]:
dates = df_topic_slice.columns[1:]
topics = df_topic_slice['topic']
data = []

for i, topic in enumerate(topics):
    data.append(go.Bar(name=topic, 
                       x=dates, 
                       y=df_topic_slice.iloc[i, 1:]))
    
fig = go.Figure(data=data)
# Change the bar mode
fig.update_layout(barmode='group')

fig.update_layout(
    font=dict(family='Courier New, monospace', size=20),
    xaxis_title='Дата',
    yaxis_title='Количество постов',
    legend_title='Темы постов',
    title=dict(text="Темы постов в ежесуточном разрезе", font=dict(size=26), automargin=True, yref='paper')
)
save_diagram(fig, 'темы_постов_в_ежесуточном_разрезе_вариант_1')
fig.show()

In [27]:
dates = df_topic_slice.columns[1:]
topics = df_topic_slice['topic']
data = []

for i, topic in enumerate(topics):
    data.append(go.Bar(name=topic, 
                       x=dates, 
                       y=df_topic_slice.iloc[i, 1:]))
    
fig = go.Figure(data=data)
# Change the bar mode
fig.update_layout(barmode='relative')

fig.update_layout(
    font=dict(family='Courier New, monospace', size=20),
    xaxis_title='Дата',
    yaxis_title='Общее количество постов',
    legend_title='Темы постов',
    title=dict(text="Темы постов в ежесуточном разрезе", font=dict(size=26), automargin=True, yref='paper')
)

save_diagram(fig, 'темы_постов_в_ежесуточном_разрезе_вариант_2')
fig.show()

В данной диаграмме высота каждого столбца равна общему количеству постов за день. <br>
По цвету выполнено разбиение каждого столбца на темы, исходя из количества постов, которые ей соответствуют.